In [3]:
TEANGA_1 = """_meta:
  text:
    type: characters
  tokens:
    base: text
    type: span
  comm:
    base: text
    type: characters
    data: string
  upos:
    data: ["ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "VERB", "X" ]
    base: tokens
    type: seq
OvM2:
  text: _Bhojpuri text_
  tokens: [[0, 7], [8, 9], [10, 20], [21, 24], [25, 26]]
  comm: lokaraṁjana ā sāṁskrtika gīta -
  upos: ["NOUN", "CCONJ", "ADJ", "NOUN", "PUNCT" ]
cN0V:
  text: _Bhojpuri text_
  tokens: [[0, 3], [4, 5], [6, 13], [14, 17], [18, 19]]
  comm: āīṁ ā saparivāra āīṁ .
  upos: ["VERB", "CCONJ", "NOUN", "VERB", "PUNCT" ]"""

CONLLU_2 = """# newdoc id = n01001
# sent_id = n01001011
# text = “While much of the digital transition is unprecedented in 
2	While	while	SCONJ	IN	_	9	mark	9:mark	_
3	much	much	ADJ	JJ	Degree=Pos	9	nsubj	9:nsubj	_
4	of	of	ADP	IN	_	7	case	7:case	_
5	the	the	DET	DT	Definite=Def|PronType=Art	7	det	7:det	_
6	digital	digital	ADJ	JJ	Degree=Pos	7	amod	7:amod	_
7	transition	transition	NOUN	NN	Number=Sing	3	nmod	3:nmod:of	_
8	is	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	9	cop	9:cop	_
9	unprecedented	unprecedented	ADJ	JJ	Degree=Pos	20	advcl	20:advcl:while	_
10	in	in	ADP	IN	_	13	case	13:case	_
"""

In [4]:
import sys

# sys.path.append('')
import teanga
import conllu

In [5]:
from conllu import parse

In [2]:
import teanga
import sys
import typing
import logging
from collections import OrderedDict

import conllu
import yaml


def read_conllu_str(s : str, db: str=None) -> teanga.Corpus:
    """Read a CoNLL-U string and return a Teanga Corpus object.

    Args:

    s: str
    The CoNLL-U string to read.
    db: str
    The DB location to use of the Teanga corpus
    """
    corpus = teanga.Corpus(db)
    corpus.



    return corpus

def read_conllu_file(file : str, db: str=None) -> teanga.Corpus:
    """Read a CoNLL-U file and return a Teanga Corpus object.

    Args:

    file: str
        The CoNLL-U file to read.
    db: str
        The DB location to use of the Teanga corpus
    """
    corpus = teanga.Corpus(db)


    return corpus


In [55]:
import teanga
import sys
import typing
import logging
from collections import OrderedDict

import conllu
import yaml

In [58]:
def read_conllu_file(file : str, db: str=None) -> teanga.Corpus:
    """Read a CoNLL-U file and return a Teanga Corpus object.

    Args:

    file: str
        The CoNLL-U file to read.
    db: str
        The DB location to use of the Teanga corpus
    """
    corpus = teanga.Corpus(db)

    #The name prefix for record keys in the output document
    OUTPUT_RECORD_NAME_PREFIX = 'sent.'

    # Placeholder value when a feature doesn't apply to a specific token:
    FEATURE_PLACEHOLDER_VALUE = None
    #FEATURE_PLACEHOLDER_VALUE = ""

    # The name prefix for feature keys in the output document
    FEATURE_NAME_PREFIX = 'feat.'

    # Token delimiters to skip (i.e. whitespaces)
    TOKEN_DELIMS = set(
        [ ' ', '\t', '\u00A0', '\u1680', '\u2005', '\u202F', '\u3000', '\uFEFF' ])


    logger = logging.getLogger(__name__)
    #logging.basicConfig(encoding='utf-8', level=logging.DEBUG)
    logging.basicConfig(encoding='utf-8', level=logging.WARNING)


    def next_span(start_offset: int, sentence: str, token: str) \
            -> typing.Tuple[typing.List[int], int]:
        """ Create the next token span
        Arguments:
            start_offset: The current sentence offset.
            sentence: The sentence text.
            token: The current token.
        Returns:
            (tuple) The span for this token,
            (int) The next starting offset.
        """

        end_offset = start_offset + len(token)
        # Note: This must be a list, not a tuple, for YAML output
        span = [ start_offset, end_offset ]

        found = sentence[start_offset:end_offset]

        logger.debug('start_offset: %s', start_offset)
        logger.debug('token: %s', token)
        logger.debug('span: %s', span)
        logger.debug('sentence[span]: %s', found)

        # Integrity assertion - Should never happen
        if found != token:
            raise ValueError(f'Token mismatch creating span for [{token}] in: {sentence}')

        # Skip over spaces
        sentence_len = len(sentence)

        while end_offset < sentence_len and sentence[end_offset] in TOKEN_DELIMS:
            logger.debug('skipping space at: i%s', end_offset)
            end_offset += 1

        if end_offset < sentence_len:
            logger.debug('next character: %s  [0x%s]',
                sentence[end_offset], sentence[end_offset].encode("utf-8").hex())

        return (span, end_offset)


    def get_token_ids(token) -> typing.Optional[typing.List[int]]:
        """ Extract the id or id range from a token
        Arguments:
            sentence: A token.
        Returns:
            A list of ids.
        """

        tid = token['id']

        # Normal token?
        if isinstance(tid, int):
            return [ tid ]

        # Verify tuple:  (int, '-' | '.', int)
        if len(tid) != 3:
            return None

        if not isinstance(tid[0], int) or not isinstance(tid[2], int):
            return None

        if tid[1] == '-':
            # Multitoken (-)
            return list(range(tid[0], tid[2] + 1))

        if tid[1] == '.':
            # empty (.)
            return [ tid[0] ]

        return None

    def process_features(token_idx: int, feats: typing.Dict[str, str],
            token_count: int, feat_lists: typing.Dict[str, typing.List[str]]) \
            -> None:
        """ Process features and save them in token position.
        Arguments:
            token_idx: The array index of this token.
            feats: The input features.
            token_count: The total number of tokens.
            feat_lists: A dictionary of output features.
        """

        for feat_name, feat_value in feats.items():
            # Get (or create) list for named feature
            try:
                feat = feat_lists[feat_name]
            except KeyError:
                feat = [ FEATURE_PLACEHOLDER_VALUE ] * token_count
                feat_lists[feat_name] = feat

            feat[token_idx] = feat_value


    def convert_sentence(sentence: typing.Sequence) -> typing.Tuple[str, dict]:
        """ Convert a CoNLL-U parsed sentence to a YAML node.
        Arguments:
            sentence: A sequence of tokens.
        Returns:
            (str) The YAML node key.
            (dict) The YAML node value.
        """

        logger.debug('sentence: %s', sentence)

        token_meta = sentence.metadata

        key = OUTPUT_RECORD_NAME_PREFIX + token_meta['sent_id']
        text = token_meta['text']

        token_count = len(sentence)

        span_list = [ None ] * token_count
        upos_list = [ None ] * token_count
        feat_lists = {}

        id_spans = {}

        span_offset = 0

        for token_idx, token in enumerate(sentence):
            logger.debug('token_idx: %s', token_idx)
            logger.debug('token.items: %s', token.items())

            ids = get_token_ids(token)

            if not ids:
                raise ValueError(f'Unexpect token type: {token}  [{token["id"]}]')

            logger.debug('ids: %s', ids)

            if ids[0] == 0:
                # Root token - use entire span
                span = [ 0, len(text) ]
            elif len(ids) == 1:
                # Normal or empty token - look for existing span
                span = id_spans.get(ids[0], None)
            else:
                span = None

            logger.debug('existing span: %s', span)

            if span:
                # Duplicate to avoid YAML refs
                span = list(span)
            else:
                # If span not already known then get the next
                (span, span_offset) = next_span(span_offset, text, token['form'])

            # Saves span for overlayed tokens
            for tid in ids:
                id_spans[tid] = span

            span_list[token_idx] = span
            upos_list[token_idx] = token['upos']

            # Process optional features
            feats = token['feats']

            if feats:
                process_features(token_idx, feats, token_count, feat_lists)

        # Build output record
        outrec = {
            'text': text,
            'tokens': span_list,
            'upos': upos_list
        }

        # Add values for each feature found in this sentence
        for feat_name, feat_values in feat_lists.items():
            outrec[FEATURE_NAME_PREFIX + feat_name] = feat_values

        return (key, outrec)




    def convert(infile: typing.TextIO, outfile: typing.TextIO) -> None:
        """ Convert a CoNLL-U file to YAML
        Arguments:
            infile: The input ".conllu" file handle.
            outfile: The output ".yaml" file handle.
        """

        sentences = conllu.parse_incr(infile)

        outdoc = OrderedDict()

        for sentence in sentences:
            (key, rec) = convert_sentence(sentence)
            outdoc[key] = rec

        yaml.dump(outdoc, outfile, sort_keys=False, allow_unicode=True)

    def convert_file(inpath: str, outpath: str) -> None:
        """ Convert a CoNLL-U file to YAML
        Arguments:
        inpath: The input ".conllu" filename.
        outpath: The output ".yaml" filename.
        """

        with open(inpath, 'r', encoding="utf-8") as infile:
            with open(outpath, 'w', encoding="utf-8") as outfile:
                convert(infile, outfile)

    def convert_all_files(files: typing.List[str]) -> None:
        """ Convert a list of CoNLL-U file(s) to YAML
        Arguments:
            fil
        es: The input ".conllu" filenames.
        """

        for inpath in files:
            outpath = inpath

            if outpath[-7:] in ( '.conllu', '.CONLLU' ):
                outpath = outpath[:-7]

        outpath += '.yaml'

        convert_file(inpath, outpath)
    #def main():
     #   if

    
    #if __name__ == '__main__':

    return corpus


In [61]:
convert_all_files(en_pud-ud-test.conllu)

NameError: name 'convert_all_files' is not defined

```python
 corpus.add_layer_meta("text")
        >>> corpus.add_layer_meta("words", layer_type="span", base="text")
        >>> corpus.add_layer_meta("pos", layer_type="seq", base="words",
        ...                        data=["NOUN", "VERB", "ADJ"])
        >>> corpus.add_layer_meta("lemma", layer_type="seq", base="words",
        ...                        data="string")
        >>> doc = corpus.add_doc("Colorless green ideas sleep furiously.")
        >>> doc.words = [(0, 9), (10, 15), (16, 21), (22, 27), (28, 37)]
        >>> doc.pos = ["ADJ", "ADJ", "NOUN", "VERB", "ADV"]
        >>> doc.lemma = ["colorless", "green", "idea", "sleep", "furiously"]
```

In [21]:
corpus = teanga.Corpus(None)

In [22]:
parsed_col = parse(CONLLU_2)

In [28]:
with open("/home/priya/teanga2/teanga/conllu/zh_patentchar-ud-test.conllu") as inpf:
    for line in inpf:
        print(line)

# global.columns = ID FORM LEMMA UPOS XPOS FEATS HEAD DEPREL DEPS MISC

# sent_id = zh_patentchar-sud-test__1

# patent_id = CN102017000573680CN00001076219020AFULZH20180123CN00F

# text = 1.一种压力传感器，所述压力传感器包括：

1	1.	_	PUNCT	_	_	5	punct	_	Gloss=_|SpaceAfter=No|Translit=1.

2	一	_	NUM	_	_	3	nummod	_	Gloss=_|SpaceAfter=No|Translit=yī

3	种	_	NUM	_	_	5	nummod	_	Gloss=_|SpaceAfter=No|Translit=zhǒng

4	压力	_	NOUN	_	_	5	nmod	_	Gloss=_|SpaceAfter=No|Translit=yā_lì

5	传感器	_	NOUN	_	_	0	root	_	Gloss=_|SpaceAfter=No|Translit=chuán_gǎn_qì

6	，	_	PUNCT	_	_	10	punct	_	Gloss=_|SpaceAfter=No|Translit=，

7	所述	_	ADJ	_	_	9	amod	_	Gloss=_|SpaceAfter=No|Translit=suǒ_shù

8	压力	_	NOUN	_	_	9	nmod	_	Gloss=_|SpaceAfter=No|Translit=yā_lì

9	传感器	_	NOUN	_	_	10	nsubj	_	Gloss=_|SpaceAfter=No|Translit=chuán_gǎn_qì

10	包括	_	VERB	_	_	5	parataxis	_	Gloss=_|SpaceAfter=No|Translit=bāo_kuò

11	：	_	PUNCT	_	_	5	punct	_	Gloss=_|SpaceAfter=No|Translit=：



# sent_id = zh_patentchar-sud-test__2

# patent_id = CN102017000573680CN0000

In [53]:
'''        >>> corpus = Corpus()
        >>> corpus.add_layer_meta("text")
        >>> corpus.add_layer_meta("words", layer_type="span", base="text")
        >>> corpus.add_layer_meta("pos", layer_type="seq", base="words",
        ...                        data=["NOUN", "VERB", "ADJ"])
        >>> doc = corpus.add_doc("Colorless green ideas sleep furiously.")
        >>> doc.words = [(0, 9), (10, 15), (16, 21), (22, 28), (29, 37)]
        >>> doc.pos = ["ADJ", "ADJ", "NOUN", "VERB", "ADV"]
        >>> corpus.val_freq("pos")
        Counter({'ADJ': 2, 'NOUN': 1, 'VERB': 1, 'ADV': 1})
        >>> corpus.val_freq("pos", ["NOUN", "VERB"])
        Counter({'NOUN': 1, 'VERB': 1})
        >>> corpus.val_freq("pos", lambda x: x[0] == "A")
        Counter({'ADJ': 2, 'ADV': 1}) '''
from teanga import Corpus
import yaml
corpus = Corpus()
corpus.add_layer_meta("text")
corpus.add_layer_meta("words", layer_type="span", base="text")
corpus.add_layer_meta("pos", layer_type="seq", base="words")

if input_type ="yml":
    with open("en_pud-ud-test.yaml") as inpf:
        docs_yml = yaml.load(inpf,Loader=yaml.Loader)
        
if input_type = "json":
    with open("en_pud-ud-test.yaml") as inpf:
        docs_yml = yaml.load(inpf,Loader=yaml.Loader)



documents = [
    {
     "text": sent_dict["text"],
     "tokens": sent_dict["tokens"], 
     "upos": sent_dict["upos"]
    }
    for (doc_id, sent_dict) in docs_yml.items()
]

for document_dict in documents:
    doc = corpus.add_doc(document_dict["text"])
    doc.words = document_dict["tokens"]
    doc.pos = document_dict["upos"]



In [54]:
for doc in corpus.docs:
    print(doc)

('5zFq', Document('5zFq', {'text': CharacterLayer('“While much of the digital transition is unprecedented in the United States, the peaceful transition of power is not,” Obama special assistant Kori Schulman wrote in a blog post Monday.'), 'words': SpanLayer([[0, 1], [1, 6], [7, 11], [12, 14], [15, 18], [19, 26], [27, 37], [38, 40], [41, 54], [55, 57], [58, 61], [62, 68], [69, 75], [75, 76], [77, 80], [81, 89], [90, 100], [101, 103], [104, 109], [110, 112], [113, 116], [116, 117], [117, 118], [119, 124], [125, 132], [133, 142], [143, 147], [148, 156], [157, 162], [163, 165], [166, 167], [168, 172], [173, 177], [178, 184], [184, 185]]), 'pos': SeqLayer(['PUNCT', 'SCONJ', 'ADJ', 'ADP', 'DET', 'ADJ', 'NOUN', 'AUX', 'ADJ', 'ADP', 'DET', 'PROPN', 'PROPN', 'PUNCT', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'AUX', 'ADV', 'PUNCT', 'PUNCT', 'PROPN', 'ADJ', 'NOUN', 'PROPN', 'PROPN', 'VERB', 'ADP', 'DET', 'NOUN', 'NOUN', 'PROPN', 'PUNCT'])}))
('79HA', Document('79HA', {'text': CharacterLayer('For thos

In [43]:
with open("en_pud-ud-test.yaml") as inpf:
    docs_yml = yaml.load(inpf,Loader=yaml.Loader)

In [51]:
docs_yml['sent.n01001011']['upos']

['PUNCT',
 'SCONJ',
 'ADJ',
 'ADP',
 'DET',
 'ADJ',
 'NOUN',
 'AUX',
 'ADJ',
 'ADP',
 'DET',
 'PROPN',
 'PROPN',
 'PUNCT',
 'DET',
 'ADJ',
 'NOUN',
 'ADP',
 'NOUN',
 'AUX',
 'ADV',
 'PUNCT',
 'PUNCT',
 'PROPN',
 'ADJ',
 'NOUN',
 'PROPN',
 'PROPN',
 'VERB',
 'ADP',
 'DET',
 'NOUN',
 'NOUN',
 'PROPN',
 'PUNCT']